In [1]:
import json
import os 
import pandas as pd
import copy

In [2]:
results_folder = "/home/studio-lab-user/all_data/vi_result"
this_folder = "kmeans_adult"
full_fname = os.path.join(results_folder, this_folder, 'results.json')

In [3]:
with open(full_fname, 'r') as f:
    this_dict = json.load(f)

In [4]:
this_dict['adult']['kmeans']['-1']['0'].keys()

dict_keys(['accs', 'nlls', 'csizes', 'times', 'wt_index'])

In [13]:
#this_dict['adult']['kmeans']['-1']['0']

In [6]:
# df columns
# dataset_name, #algorithm, coreset_size, accuracy, nll, times

In [5]:
df_list = []
for dataset in this_dict.keys():
    for algo in this_dict[dataset].keys():
        for trial_num in this_dict[dataset][algo]['-1'].keys():
            final_dict = this_dict[dataset][algo]['-1'][trial_num]
            all_acc = final_dict['accs']
            all_nll = final_dict['nlls']
            all_csizes = final_dict['csizes']
            for (size, acc, nll) in zip(all_csizes, all_acc, all_nll):
                df_list.append({
                    'dataset': dataset,
                    'algo': algo,
                    'coreset_size': size,
                    'accuracy': acc,
                    'nll': nll
                })

result_df = pd.DataFrame(df_list)
            
result_df.head()        

NameError: name 'this_dict' is not defined

In [20]:
# this is the case, where only key is -1, and 
# coreset sizes are contained in the log_every epochs
import copy 

def process_unconstrained_data(this_dict, dataset, algo, config_dict):
    df_list = []
    for trial_num in this_dict[dataset][algo]['-1'].keys():
        final_dict = this_dict[dataset][algo]['-1'][trial_num]
        all_acc = final_dict['accs']
        all_nll = final_dict['nlls']
        all_csizes = final_dict['csizes']
        for (size, acc, nll) in zip(all_csizes, all_acc, all_nll):
            coreset_dict = {
                'dataset': dataset,
                'algo': algo,
                'coreset_size': size,
                'accuracy': acc,
                'nll': nll,
                'trial_num': trial_num
            }
            
            coreset_dict.update(config_dict)
            df_list.append(coreset_dict)
            
    return df_list

# each key is a coreset size here, and the key is not -1
def process_constrained_data(this_dict, dataset, algo, config_dict):
    df_list = []
    for csize in this_dict[dataset][algo].keys():
        for trial_num in this_dict[dataset][algo][csize].keys():
            final_dict = this_dict[dataset][algo][csize][trial_num]
            acc = final_dict['accs'][-1]
            nll = final_dict['nlls'][-1]
            coreset_dict = {
                'dataset': dataset,
                'algo': algo,
                'coreset_size': csize,
                'accuracy': acc,
                'nll': nll,
                'trial_num': trial_num
            }
            
            coreset_dict.update(config_dict)
            df_list.append(coreset_dict)
            
    return df_list



def process_result_df(results_folder, subfolder_list):
    df_list = []
    
    for subfolder in subfolder_list:
        full_fname = os.path.join(results_folder, subfolder, 'results.json')
        config_fname = os.path.join(results_folder, subfolder, 'config.json')
        
        with open(full_fname, 'r') as f:
            this_dict = json.load(f)
        
        # handle config file
        subset_keys = ['mc_samples', 'num_epochs', 'inner_it', 'outer_it',
                       'lr0net', 'mfvi_selection_method']
        

        if os.path.exists(config_fname):
            with open(config_fname, 'r') as f:
                config_dict = json.load(f)

            select_config = {k:v for k,v in config_dict.items() if k in subset_keys}
            
        else:
            select_config = {k:0 for k in subset_keys}
            

        for dataset in this_dict.keys():
            for algo in this_dict[dataset].keys():
                key_list = list(this_dict[dataset][algo].keys())
                select_config_copy = copy.deepcopy(select_config)
                
                if '-1' in key_list: 
                    df_list += process_unconstrained_data(
                        this_dict=this_dict, dataset=dataset, algo=algo,
                        config_dict=select_config_copy
                    )
                else:
                    df_list += process_constrained_data(
                        this_dict=this_dict, dataset=dataset, algo=algo,
                        config_dict=select_config_copy
                    )

    result_df = pd.DataFrame(df_list)
    
    return result_df


In [4]:
subfolder_list = [
    'mfvi_subset_webspam_2023_03_27_22_59_21',
    'mfvi_subset_webspam_2023_03_27_23_44_56',
    'kmeans_adult'
]
        

In [5]:
full_df = process_result_df(
    results_folder=results_folder,
    subfolder_list=subfolder_list
)

In [6]:
full_df

,dataset,algo,coreset_size,accuracy,nll,mc_samples,num_epochs,inner_it,outer_it,lr0net,mfvi_selection_method
0,webspam,mfvi_subset,40,0.819382,0.406128,256,50,1000,100,0.01,kmeans
1,webspam,mfvi_subset,80,0.852923,0.383030,256,50,1000,100,0.01,kmeans
2,webspam,mfvi_subset,40,0.819382,0.406128,256,50,1000,100,0.01,kmeans
3,webspam,mfvi_subset,40,0.770585,0.480500,256,50,1000,100,0.01,kmeans
4,webspam,mfvi_subset,40,0.838752,0.406806,256,50,1000,100,0.01,kmeans
5,webspam,mfvi_subset,80,0.852923,0.383030,256,50,1000,100,0.01,kmeans
6,webspam,mfvi_subset,80,0.809497,0.417634,256,50,1000,100,0.01,kmeans
7,webspam,mfvi_subset,80,0.834409,0.388863,256,50,1000,100,0.01,kmeans
8,adult,kmeans,4,0.518128,0.691310,0,0,0,0,0.00,0
9,adult,kmeans,10,0.589841,1.606101,0,0,0,0,0.00,0


In [21]:
def get_all_subfolders(directory):
    contents = os.listdir(directory)
    subfolder_list = []

    # loop through the list of contents and print each folder name
    for item in contents:
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            subfolder_list.append(item)
    
    return subfolder_list

In [22]:
get_all_subfolders(results_folder)

['.ipynb_checkpoints',
 'random_phishing',
 'random_sparsevi_webspam',
 'random_webspam',
 'sparsevi_webspam',
 'kmeans_webspam',
 'kmeans_phishing',
 'kmeans_adult',
 'mfvi_subset_webspam_2023_03_27_22_59_21',
 'mfvi_subset_webspam_2023_03_27_23_44_56',
 'mfvi_selection_webspam_2023_03_28_02_39_13',
 'mfvi_selection_webspam_2023_03_28_02_40_24',
 'mfvi_selection_webspam_2023_03_28_02_45_21',
 'mfvi_selection_webspam_2023_03_28_02_48_24',
 'mfvi_selection_webspam_2023_03_28_02_50_41',
 'mfvi_selection_webspam_2023_03_28_02_52_59',
 'mfvi_selection_webspam_2023_03_28_03_14_51',
 'mfvi_selection_webspam_2023_03_28_03_36_41',
 'mfvi_selection_webspam_2023_03_28_03_58_34',
 'mfvi_selection_webspam_2023_03_28_04_22_59',
 'mfvi_selection_webspam_2023_03_28_04_47_23',
 'mfvi_selection_webspam_2023_03_28_05_10_35',
 'mfvi_selection_webspam_2023_03_28_05_33_14',
 'mfvi_selection_webspam_2023_03_28_05_53_48',
 'mfvi_selection_webspam_2023_03_28_06_16_37',
 'mfvi_selection_webspam_2023_03_28_06_4

In [23]:
all_folders = get_all_subfolders(results_folder)
subfolder_list = [x for x in all_folders if '2023_03_28' in x] 
full_df = process_result_df(
    results_folder=results_folder,
    subfolder_list=subfolder_list
)

In [24]:
full_df.query('coreset_size=="100"').sort_values(by='accuracy', ascending=False).head()

,dataset,algo,coreset_size,accuracy,nll,trial_num,mc_samples,num_epochs,inner_it,outer_it,lr0net,mfvi_selection_method
14,webspam,mfvi_selection,100,0.886064,0.316236,2,256,200,20,100,0.01,kmeans
140,webspam,mfvi_selection,100,0.886064,0.316236,2,256,200,20,100,0.01,scored_kmeans_forgetting
12,webspam,mfvi_selection,100,0.867379,0.327906,0,256,200,20,100,0.01,kmeans
138,webspam,mfvi_selection,100,0.867379,0.327906,0,256,200,20,100,0.01,scored_kmeans_forgetting
31,webspam,mfvi_selection,100,0.859665,0.328534,1,256,200,20,100,0.01,random


In [25]:
full_df.query('coreset_size=="40"').sort_values(by='accuracy', ascending=False).head()

,dataset,algo,coreset_size,accuracy,nll,trial_num,mc_samples,num_epochs,inner_it,outer_it,lr0net,mfvi_selection_method
28,webspam,mfvi_selection,40,0.846466,0.391521,1,256,200,20,100,0.010,random
9,webspam,mfvi_selection,40,0.827896,0.388697,0,256,200,20,100,0.010,kmeans
135,webspam,mfvi_selection,40,0.827896,0.388697,0,256,200,20,100,0.010,scored_kmeans_forgetting
37,webspam,mfvi_selection,40,0.825267,0.482932,1,256,200,20,100,0.001,random
10,webspam,mfvi_selection,40,0.813668,0.441609,1,256,200,20,100,0.010,kmeans


In [26]:
full_df.query('coreset_size=="10"').sort_values(by='accuracy', ascending=False).head()

,dataset,algo,coreset_size,accuracy,nll,trial_num,mc_samples,num_epochs,inner_it,outer_it,lr0net,mfvi_selection_method
33,webspam,mfvi_selection,10,0.759842,0.519736,0,256,200,20,100,0.001,random
34,webspam,mfvi_selection,10,0.744129,0.544270,1,256,200,20,100,0.001,random
25,webspam,mfvi_selection,10,0.733330,0.628162,1,256,200,20,100,0.010,random
151,webspam,mfvi_selection,10,0.724530,0.495100,1,256,200,20,100,0.010,scored_kmeans_entropy
170,webspam,mfvi_selection,10,0.687961,1.534839,2,256,200,20,100,0.010,scored_kmeans_least_confidence


### Prod Results

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from collections import defaultdict
import os 
import pickle 
import json

In [34]:
def rec_dd():
    return defaultdict(rec_dd)

#dataset_name = 'MNIST'
#subfolder_name = 'psvi_alpha_fixed_u_MNIST_2023_04_18_19_37_52'

def process_dataset(subfolder_name, dataset_name, method_name):
    RESULTS_FOLDER = '/home/studio-lab-user/all_data/vi_result'

    fname = os.path.join(RESULTS_FOLDER, subfolder_name , 'results.pk')
    json_fname = os.path.join(RESULTS_FOLDER, subfolder_name, 'config.json')
    with open(fname, 'rb') as f:
        results = pickle.load(f)

    with open(json_fname, 'r') as f:
        config = json.load(f)

    method = config['methods'][0]
    coreset_sizes = config['coreset_sizes']
    mfvi_selection_method = config['mfvi_selection_method']
    init_at = config['init_at']
    num_trials = config['num_trials']

    select_method = 'random' if init_at == 'subsample' else mfvi_selection_method

    df_list = []

    
    for sz in coreset_sizes:
        all_trial_acc = []
        for trial in range(num_trials):
            acc_list = results[dataset_name][method_name][sz][trial]['accs']
            max_acc = max(acc_list)
            all_trial_acc.append(max_acc)
        acc_mean = np.mean(all_trial_acc)
        acc_std = np.std(all_trial_acc)
        
        df_list.append({
            'method': method,
            'coreset_size': sz,
            'selection': select_method,
            'accuracy': acc_mean,
            'accuracy_std': acc_std,
            'dataset': dataset_name
        })

    df = pd.DataFrame(df_list)
    return df 

def get_all_subfolders(directory):
    contents = os.listdir(directory)
    subfolder_list = []

    # loop through the list of contents and print each folder name
    for item in contents:
        item_path = os.path.join(directory, item)
        if os.path.isdir(item_path):
            subfolder_list.append(item)
    
    return subfolder_list

def get_subfolders_for_date(date_str):
    RESULTS_FOLDER = '/home/studio-lab-user/all_data/vi_result'
    all_folders = get_all_subfolders(RESULTS_FOLDER)
    subfolder_list = [x for x in all_folders if date_str in x] 
    return subfolder_list 


def custom_pivot_result(df):
    coreset_sizes = df['coreset_size'].unique()
    methods = df['selection'].unique()

    dict_list = []

    for mth in methods:
        method_dict = {str(x): 0 for x in coreset_sizes}
        method_dict['method'] = mth
        for sz in coreset_sizes:
            acc_mean = df.query(f"coreset_size=={sz} and selection=='{mth}'").iloc[0]['accuracy']
            acc_std = df.query(f"coreset_size=={sz} and selection=='{mth}'").iloc[0]['accuracy_std']
            method_dict[str(sz)] = f"{np.round(acc_mean, 4)} +- {np.round(acc_std, 2)}" 
        dict_list.append(method_dict)
    final_df = pd.DataFrame(dict_list)
    return final_df 


def combine_folder_results(subfolder_list, dataset_name, method_name):
    """
    Forms the final table, combining the results from multiple subfolders
    """
    df_list = []
    for subfolder in subfolder_list: 
        df = process_dataset(subfolder, dataset_name, method_name)
        df_list.append(df)
    
    final_df = pd.concat(df_list)
    return final_df 


In [35]:
#all_folders = get_all_subfolders('.')
get_subfolders_for_date('2023_04_26')


['psvi_evaluate_MNIST_2023_04_26_22_41_40',
 'psvi_evaluate_MNIST_2023_04_26_23_57_17']

In [36]:
 df = combine_folder_results(
     subfolder_list=[
         'psvi_evaluate_MNIST_2023_04_26_22_41_40',
         'psvi_evaluate_MNIST_2023_04_26_23_57_17',
     ],
     dataset_name='MNIST',
     method_name = 'psvi_evaluate'
 )
df.head()

,method,coreset_size,selection,accuracy,accuracy_std,dataset
0,psvi_evaluate,30,random,0.704733,0.014087,MNIST
1,psvi_evaluate,50,random,0.797300,0.015558,MNIST
2,psvi_evaluate,80,random,0.853433,0.005414,MNIST
3,psvi_evaluate,100,random,0.872267,0.006274,MNIST
0,psvi_evaluate,30,kmeans,0.737867,0.006698,MNIST


In [37]:
custom_pivot_result(df)

,30,50,80,100,method
0,0.7047 +- 0.01,0.7973 +- 0.02,0.8534 +- 0.01,0.8723 +- 0.01,random
1,0.7379 +- 0.01,0.8069 +- 0.0,0.8359 +- 0.01,0.8603 +- 0.0,kmeans


In [21]:
df

,method,coreset_size,selection,accuracy,dataset
0,psvi_evaluate,30,random,0.6902,MNIST
1,psvi_evaluate,50,random,0.7753,MNIST
2,psvi_evaluate,80,random,0.8604,MNIST
3,psvi_evaluate,100,random,0.8634,MNIST
0,psvi_evaluate,30,kmeans,0.7376,MNIST
1,psvi_evaluate,50,kmeans,0.8064,MNIST
2,psvi_evaluate,80,kmeans,0.8503,MNIST
3,psvi_evaluate,100,kmeans,0.8641,MNIST
